# Regresja logistyczna i SVM

Opis regresji logistycznej autorstwa Magdaleny Trędowicz został zapożyczony z zajęć Olimpijskiego Koła Sztucznej Inteligencji na Uniwersytecie Jagiellońskim.

In [1]:
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_moons

## Regresja logistyczna
Załóżmy, że każdy przykład ze zbioru danych $x_i \in X$ ma przypisaną etykietę $y_i \in \{1, \ldots, K \}$.
Regresja logistyczna jest jednym z klasycznych modeli, który bezpośrednio nadaje się zarówno do klasyfikacji binarnej (dwie klasy: $y_i \in \{0,1\}$), jak i wieloklasowej. 
Ten model jest o tyle ważny, że stanowi podstawę modeli klasyfikacyjnych opartych o sieci neuronowe. 

Regresja logistyczna tworzy model probabilistyczny określający prawdopodobieństwo przynależności punktu do poszczególnych klas. 
W tym modelu chcemy dla każdego punktu wyznaczyć tak zwany rozkład a posteriori $p(1|x_i),\ldots,p(K|x_i)$ określający przynależność punktu $x_i$ do każdej z $K$ klas. Jako finalną decyzję o klasyfikacji przyjmujemy tę najbardziej prawdopodobną, czyli:

$$
c(x) = \arg \max_{k \in \{1, \dots, K\}} p(k | x_i).
$$

Żeby zbudować taki model, musimy sparametryzować prawdopodobieństwa a posteriori, a następnie zbudować funkcję kosztu definiującą kryterium optymalizacyjne. W celu sparametryzowania $p(k|\cdot)$, określmy moc przyporządkowania punktu $x$ do klasy $k$, wykorzystując model liniowy postaci:

$$
f_k(x) = w_k^Tx+b_k \in \mathbb{R}, \quad \text{dla} \quad k \in \{1, \ldots, K\}.
$$

Mając $K$ takich modeli liniowych, możemy wskazać najbardziej prawdopodobną klasę poprzez wyznaczenie największej wartości $f_k(x)$ dla $k \in \{1, \ldots, K\}$.
W celu transformacji tcyh funkcji do prawdopodobieństw wykorzystamy funkcję **softmax** postaci:

$$
p(k|x) = \frac{\exp(z_k)}{\sum_{j=1}^{K}\exp(z_j)} \in (0,1),
$$

gdzie $z_k = f_k(x) = w_k^Tx + b_k.$

### Zadanie 1 (4 punkty)
Wyucz model regresji logistycznej na zbiorze danych Iris, tak że:
- Klasyfikator używa tylko jednej zmiennej - szerokości płatka (petal width) oraz wykrywa tylko jedną klasę - Virginica.
- Podziel dane na zbiór treningowy, walidacyjny i testowy. Następnie zrób grid search w celu dobrania optymalnego hiperparametru $C$ (jego opis można znaleźć tutaj: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). Grid search powinien uwzględnić przynajmniej 10 różnych wartości $C$.
- Narysuj wykres przedstawiący dokładność klasyfikatora na zbiorze testowym w zależności od wartości hiperparametru $C$. 
- Narysuj wykres przedstawiający granicę decyzyjną klasyfikatora. Na osi OX powinna być długość płatka (petal width), a na osi OY prawdopodobieństwo przynależności kwiatu do danej klasy. Taki wykres powinien być narysowany w oparciu o dane testowe.

Nie zapomnij podpisać wykresów oraz dodać legendy!

In [4]:
iris = datasets.load_iris()
list(iris.keys())

['data',
 'target',
 'frame',
 'target_names',
 'DESCR',
 'feature_names',
 'filename',
 'data_module']

In [6]:
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

### Zadanie nr 2 (1 punkt)
Powtórz zadanie nr 1, ale tym razem weź pod uwagę dwie zmienne - szerokość i długość płatka (petal width, petal length). Zwizualizuj granicę decyzyjną klasyfikatora.

## Maszyny Wektorów Nośnych (SVM)

Maszyny Wektorów Nośnych (SVM, ang. Support Vector Machines) to modele nadzorowanego uczenia maszynowego, używane do klasyfikacji i regresji. Główna idea polega na znalezieniu **hiperpłaszczyzny**, która oddziela dane różnych klas z **maksymalnym marginesem**.

### Liniowe SVM dla klasyfikacji binarnej

Dla zbioru treningowego $ \{ (\mathbf{x}_i, y_i) \}_{i=1}^n $, gdzie $ \mathbf{x}_i \in \mathbb{R}^d $ oraz $ y_i \in \{-1, 1\} $, celem jest znalezienie hiperpłaszczyzny:

$$
f(\mathbf{x}) = \mathbf{w}^\top \mathbf{x} + b = 0
$$

która spełnia warunek:

$$
y_i (\mathbf{w}^\top \mathbf{x}_i + b) \geq 1, \quad \forall i
$$

Problem optymalizacyjny ma postać:

$$
\min_{\mathbf{w}, b} \ \frac{1}{2} \|\mathbf{w}\|^2 \quad \text{przy założeniu} \quad y_i (\mathbf{w}^\top \mathbf{x}_i + b) \geq 1
$$

Jest to problem wypukłej optymalizacji kwadratowej. Maksymalizujemy margines $ \frac{2}{\|\mathbf{w}\|} $ między najbliższymi punktami klas, czyli tzw. **wektorami nośnymi**.

### Miękki margines (Soft Margin SVM)

Aby poradzić sobie z danymi, które nie są liniowo separowalne, wprowadza się tzw. slack variables $ \xi_i \geq 0 $:

$$
\min_{\mathbf{w}, b, \boldsymbol{\xi}} \ \frac{1}{2} \|\mathbf{w}\|^2 + C \sum_{i=1}^n \xi_i \quad \text{przy założeniu} \quad y_i (\mathbf{w}^\top \mathbf{x}_i + b) \geq 1 - \xi_i
$$

gdzie $ C > 0 $ to parametr regularyzacji, który równoważy kompromis między szerokością marginesu a karą za błędy klasyfikacji.

### Postać dualna

SVM często rozwiązuje się w postaci dualnej. Problem dualny ma postać:

$$
\max_{\boldsymbol{\alpha}} \sum_{i=1}^n \alpha_i - \frac{1}{2} \sum_{i,j=1}^n \alpha_i \alpha_j y_i y_j \langle \mathbf{x}_i, \mathbf{x}_j \rangle
\quad \text{przy założeniach} \quad
0 \leq \alpha_i \leq C, \quad \sum_{i=1}^n \alpha_i y_i = 0
$$

Tylko te obserwacje, dla których $ \alpha_i > 0 $, stają się **wektorami nośnymi**.

---

## Sztuczka Jądra (Kernel Trick)

Gdy dane nie są liniowo separowalne w przestrzeni wejściowej $ \mathbb{R}^d $, można je przekształcić do przestrzeni cech o wyższej wymiarowości przy pomocy odwzorowania $ \phi: \mathbb{R}^d \rightarrow \mathcal{H} $. Jednak obliczanie $ \phi(\mathbf{x}) $ może być kosztowne lub niewykonalne.

**Sztuczka jądra** pozwala tego uniknąć, definiując funkcję jądra $ K $ taką, że:

$$
K(\mathbf{x}_i, \mathbf{x}_j) = \langle \phi(\mathbf{x}_i), \phi(\mathbf{x}_j) \rangle
$$

Dzięki temu postać dualna może być wyrażona tylko przy pomocy funkcji jądra:

$$
\max_{\boldsymbol{\alpha}} \sum_{i=1}^n \alpha_i - \frac{1}{2} \sum_{i,j=1}^n \alpha_i \alpha_j y_i y_j K(\mathbf{x}_i, \mathbf{x}_j)
$$

### Przykładowe funkcje jądra

1. **Jądro liniowe**:
$$
K(\mathbf{x}, \mathbf{z}) = \mathbf{x}^\top \mathbf{z}
$$

2. **Jądro wielomianowe**:
$$
K(\mathbf{x}, \mathbf{z}) = (\mathbf{x}^\top \mathbf{z} + c)^d
$$

3. **Jądro RBF / Gaussowskie**:
$$
K(\mathbf{x}, \mathbf{z}) = \exp\left(-\frac{\|\mathbf{x} - \mathbf{z}\|^2}{2\sigma^2}\right)
$$

4. **Jądro sigmoidalne** (związane z sieciami neuronowymi):
$$
K(\mathbf{x}, \mathbf{z}) = \tanh(\kappa \mathbf{x}^\top \mathbf{z} + \theta)
$$

### Funkcja decyzyjna

Po nauczeniu modelu, funkcja decyzyjna dla nowego punktu $ \mathbf{x} $ to:

$$
f(\mathbf{x}) = \text{sign} \left( \sum_{i=1}^n \alpha_i y_i K(\mathbf{x}_i, \mathbf{x}) + b \right)
$$

Tylko wektory nośne mają wpływ na wartość tej funkcji, co sprawia, że predykcja w SVM jest wydajna obliczeniowo.

### Zadanie 3 (5 punktów)

Zbadaj, jak różne jądra (kernel functions) i ich parametry wpływają na jakość klasyfikacji oraz kształt granicy decyzyjnej w klasyfikacji danych make_moons:
- Podziel dane na treningowe, testowe i walidacyjne.
- Zrób grid searcha po hiperparameterach, by znaleźć te najlepsze. Grid search powinien być zrobiony dla jądra: liniowego, wielomianowego, RBF i sigmoidalnego.
- Wykorzstaj metryki: F1 score, precision oraz accuracy do oceny jakości modelu.
- Dla każdego jądra z najlepszymi hiperparametrami zwizualizuj granicę decyzyjną.


In [ ]:
# Dane do zadania
moons_data = make_moons(n_samples=300, noise=0.25, random_state=42)